In [17]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

## Reading in data for each lake

### Lake Superior

In [21]:
####### EVAPORATION #######

# Read the csv file and make 2000 the first year
filePath = '../Data/evaporation_sup.csv'
s_evap = pd.read_csv(filePath, skiprows=3)
s_evap = s_evap.iloc[50:, :]

# Adjsuting the way data frame is presented
## we want two columns: Time and Evap Value

## melt the data frame and make 'Month' a categorical variable
s_evap_melted = pd.melt(s_evap, id_vars=['YYYY'], var_name='Month', value_name='Evap Value')
monthOrder = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
s_evap_melted['Month'] = pd.Categorical(s_evap_melted['Month'], categories=monthOrder, ordered=True)

## now sort the DataFrame by 'Year' and 'Month'
s_evap_melted = s_evap_melted.sort_values(by=['YYYY', 'Month'])

## combine Month and Year into one column
s_evap_melted['Time'] = s_evap_melted['Month'].astype(str) + '-' + s_evap_melted['YYYY'].astype(str)

## create the final 1D DataFrame with two desired columns and evaporation values
s_evap = s_evap_melted[['Time', 'Evap Value']].reset_index(drop=True)

# Conversion
s_evap['Evap Value'] = s_evap['Evap Value'] * 0.001

# s_evap

In [22]:
####### PRECIPITATION #######

# Read the csv file and make 2000 the first year
filePath = '../Data/prc_sup_lake_mon.csv'
s_prc = pd.read_csv(filePath, skiprows=5)
s_prc = s_prc.iloc[60:81, :]

# Adjsuting the way data frame is presented
## we want two columns: Time and Prc Value

## melt the data frame and make 'Month' a categorical variable
s_prc_melted = pd.melt(s_prc, id_vars=['YYYY'], var_name='Month', value_name='Prc Value')
monthOrder = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
s_prc_melted['Month'] = pd.Categorical(s_prc_melted['Month'], categories=monthOrder, ordered=True)

## now sort the DataFrame by 'Year' and 'Month'
s_prc_melted = s_prc_melted.sort_values(by=['YYYY', 'Month'])

## combine Month and Year into one column
s_prc_melted['Time'] = s_prc_melted['Month'].astype(str) + '-' + s_prc_melted['YYYY'].astype(str)

# create the final 1D DataFrame with two desired columns and prc values
s_prc = s_prc_melted[['Time', 'Prc Value']].reset_index(drop=True)
s_prc['Prc Value'] = s_prc['Prc Value'].astype(float)

# Conversion
s_prc['Prc Value'] = s_prc['Prc Value'] * 0.001

# s_prc

In [122]:
def get_time(month_year: str) -> float:
    '''
    Conversion flow in m^3/s to m^3/month
    '''
    month = month_year[0:3]
    if month in ['Apr', 'Jun', 'Sep', 'Nov']:
        return 2.592 * 1e6
    elif month in ['Jan', 'Mar', 'May', 'Jul', 'Aug', 'Oct', 'Dec']:
        return 2.6784*1e6
    else:
        return 2.4192*1e6

In [123]:
####### RUNOFF #######

# Read the csv file and make 2000 the first year
filePath = '../Data/runoff_eri_arm.csv'
s_runoff = pd.read_csv(filePath, skiprows=2)
s_runoff = s_runoff.iloc[1224:, :3]
s_runoff = s_runoff.rename(columns={'Mon': 'Month'})
s_runoff = s_runoff.rename(columns={'Flow': 'Runoff Value (Flow)'})

# Adjsuting the way data frame is presented
## we want two columns: Time and Runoff Value

## melt the data frame and make 'Month' a categorical variable
month_conversion = {1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun', 7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'}
s_runoff['Month'] = s_runoff['Month'].map(month_conversion)

## combine Month and Year into one column
s_runoff['Time'] = s_runoff['Month'].astype(str) + '-' + s_runoff['Year'].astype(str)

# # create the final 1D DataFrame with two desired columns and runoff values
s_runoff = s_runoff[['Time', 'Runoff Value (Flow)']].reset_index(drop=True)
s_runoff = s_runoff.iloc[:-4, :]

# Conversion

s_SA_inv = (1 / (82.103 * 1e9 ))

s_runoff['Runoff Value (Flow)'] = s_runoff['Runoff Value (Flow)'].apply(lambda x: x * s_SA_inv)

s_runoff['Runoff Value (Flow)'] *= s_runoff['Time'].apply(get_time)

In [124]:
s_runoff['Runoff Value (Flow)']

0      0.011596
1      0.021817
2      0.015963
3      0.032691
4      0.028294
         ...   
243    0.027618
244    0.043492
245    0.009093
246    0.005892
247    0.005527
Name: Runoff Value (Flow), Length: 248, dtype: float64

In [125]:
s_superior = s_runoff.copy()
s_superior['Runoff Value (Flow)'] = s_prc.iloc[0:248, 1] - s_evap.iloc[0:248, 1] + s_runoff['Runoff Value (Flow)']
s_superior.rename(columns={'Runoff Value (Flow)':'Value'}, inplace=True)

In [126]:
s_superior

,Time,Value
0,Jan-2000,-0.044614
1,Feb-2000,-0.013353
2,Mar-2000,0.040233
3,Apr-2000,0.039931
4,May-2000,0.084614
...,...,...
243,Apr-2020,0.055828
244,May-2020,0.078782
245,Jun-2020,0.084413
246,Jul-2020,0.123642


### Lakes Michigan and Huron

Note: since we consider these two lakes as one entity, we will add their evaporation, precipitation, and runoff data

In [114]:
####### EVAPORATION #######

def evaporation_data(file_path):
    
    # Read the csv file and make 2000 the first year
    evap = pd.read_csv(file_path, skiprows=3)
    evap = evap.iloc[50:, :]

    # Adjsuting the way data frame is presented
    ## we want two columns: Time and Prc Value

    ## melt the data frame and make 'Month' a categorical variable
    evap_melted = pd.melt(evap, id_vars=['YYYY'], var_name='Month', value_name='Evap Value')
    month_order = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    evap_melted['Month'] = pd.Categorical(evap_melted['Month'], categories=month_order, ordered=True)

    ## now sort the DataFrame by 'Year' and 'Month'
    evap_melted = evap_melted.sort_values(by=['YYYY', 'Month'])

    ## combine Month and Year into one column
    evap_melted['Time'] = evap_melted['Month'].astype(str) + '-' + evap_melted['YYYY'].astype(str)

    # Create the final 1D DataFrame with two desired columns and prc values
    evap = evap_melted[['Time', 'Evap Value']].reset_index(drop=True)
    
    # Conversion
    evap['Evap Value'] = evap['Evap Value'] * 0.001

    return evap


# Apply the function to Lakes Michigan and Huron
file_path_h = '../Data/evaporation_hur.csv'
h_evap_data = evaporation_data(file_path_h)

file_path_m = '../Data/evaporation_mic.csv'
m_evap_data = evaporation_data(file_path_m)

# Combine two DataFrames, as these two lakes are considered to be one entity in our analysis
mh_evap_data = m_evap_data + h_evap_data
mh_evap_data['Time'] = m_evap_data['Time']

# mh_evap_data

In [115]:
####### PRECIPITATION #######

def precipitation_data(file_path, start_row=5, start_index=60, end_index=81):
    
    # Read the csv file and make 2000 the first year
    prc = pd.read_csv(file_path, skiprows=start_row)
    prc = prc.iloc[start_index:end_index, :]

    # Adjsuting the way data frame is presented
    ## we want two columns: Time and Prc Value

    ## melt the data frame and make 'Month' a categorical variable
    prc_melted = pd.melt(prc, id_vars=['YYYY'], var_name='Month', value_name='Prc Value')
    month_order = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    prc_melted['Month'] = pd.Categorical(prc_melted['Month'], categories=month_order, ordered=True)

    ## now sort the DataFrame by 'Year' and 'Month'
    prc_melted = prc_melted.sort_values(by=['YYYY', 'Month'])

    ## combine Month and Year into one column
    prc_melted['Time'] = prc_melted['Month'].astype(str) + '-' + prc_melted['YYYY'].astype(str)

    # Create the final 1D DataFrame with two desired columns and prc values
    prc = prc_melted[['Time', 'Prc Value']].reset_index(drop=True)
    prc['Prc Value'] = prc['Prc Value'].astype(float)

    # Conversion
    prc['Prc Value'] = prc['Prc Value'] * 0.001

    return prc


# Apply the function to Lakes Michigan and Huron
file_path_h = '../Data/prc_hur_lake_mon.csv'
h_prc_data = precipitation_data(file_path_h)

file_path_m = '../Data/prc_mic_lake_mon.csv'
m_prc_data = precipitation_data(file_path_m)

# Combine two DataFrames, as these two lakes are considered to be one entity in our analysis
mh_prc_data = m_prc_data + h_prc_data
mh_prc_data['Time'] = m_prc_data['Time']

# mh_prc_data

In [127]:
####### RUNOFF #######

def runoff_data(file_path):
    
    # Read the CSV file and make 2000 the first year
    runoff = pd.read_csv(file_path, skiprows=2)
    runoff = runoff.iloc[1224:, :3]
    runoff = runoff.rename(columns={'Mon': 'Month'})
    runoff = runoff.rename(columns={'Flow': 'Runoff Value (Flow)'})
    
    # Adjust the way data frame is presented
    ## we want two columns: Time and Runoff Value

    ## melt the data frame and make 'Month' a categorical variabl
    month_conversion = {1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun', 7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'}
    runoff['Month'] = runoff['Month'].map(month_conversion)

    ## combine Month and Year into one column
    runoff['Time'] = runoff['Month'].astype(str) + '-' + runoff['Year'].astype(str)

    # Create the final 1D DataFrame with two desired columns and runoff values
    runoff = runoff[['Time', 'Runoff Value (Flow)']].reset_index(drop=True)
    runoff = runoff.iloc[:-4, :]

    # Conversion
    # ...

    return runoff


# Apply the function to Lakes Michigan and Huron
file_path_h = '../Data/runoff_hur_arm.csv'
h_runoff_data = runoff_data(file_path_h)

file_path_m = '../Data/runoff_mic_arm.csv'
m_runoff_data = runoff_data(file_path_m)

# Combine two DataFrames, as these two lakes are considered to be one entity in our analysis
mh_runoff_data = m_runoff_data + h_runoff_data
mh_runoff_data['Time'] = m_runoff_data['Time']

mh_SA_inv = (1 / (117.63 * 1e9 ))

mh_runoff_data['Runoff Value (Flow)'] = mh_runoff_data['Runoff Value (Flow)'].apply(lambda x: x * mh_SA_inv)

mh_runoff_data['Runoff Value (Flow)'] *= mh_runoff_data['Time'].apply(get_time)

In [128]:
s_mh = mh_runoff_data.copy()
s_mh['Runoff Value (Flow)'] = mh_prc_data.iloc[0:248, 1] - mh_evap_data.iloc[0:248, 1] + mh_runoff_data['Runoff Value (Flow)']
s_mh.rename(columns={'Runoff Value (Flow)':'Value'}, inplace=True)

In [129]:
s_mh

,Time,Value
0,Jan-2003,-0.063990
1,Feb-2003,0.038968
2,Mar-2003,0.067264
3,Apr-2003,0.145403
4,May-2003,0.298774
...,...,...
207,Apr-2020,0.288399
208,May-2020,0.220694
209,Jun-2020,0.294964
210,Jul-2020,0.173910


### Lake Erie

In [120]:
####### EVAPORATION #######

# Read the csv file and make 2000 the first year
filePath = '../Data/evaporation_eri.csv'
e_evap = pd.read_csv(filePath, skiprows=3)
e_evap = e_evap.iloc[50:, :]

# Adjsuting the way data frame is presented
## we want two columns: Time and Evap Value

## melt the data frame and make 'Month' a categorical variable
e_evap_melted = pd.melt(e_evap, id_vars=['YYYY'], var_name='Month', value_name='Evap Value')
monthOrder = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
e_evap_melted['Month'] = pd.Categorical(e_evap_melted['Month'], categories=monthOrder, ordered=True)

## now sort the DataFrame by 'Year' and 'Month'
e_evap_melted = e_evap_melted.sort_values(by=['YYYY', 'Month'])

## combine Month and Year into one column
e_evap_melted['Time'] = e_evap_melted['Month'].astype(str) + '-' + e_evap_melted['YYYY'].astype(str)

## create the final 1D DataFrame with two desired columns and evaporation values
e_evap = e_evap_melted[['Time', 'Evap Value']].reset_index(drop=True)

# Conversion
e_evap['Evap Value'] = e_evap['Evap Value'] * 0.001

# e_evap


In [121]:
####### PRECIPITATION #######

# Read the csv file and make 2000 the first year
filePath = '../Data/prc_eri_lake_mon.csv'
e_prc = pd.read_csv(filePath, skiprows=5)
e_prc = e_prc.iloc[60:81, :]

# Adjsuting the way data frame is presented
## we want two columns: Time and Prc Value

## melt the data frame and make 'Month' a categorical variable
e_prc_melted = pd.melt(e_prc, id_vars=['YYYY'], var_name='Month', value_name='Prc Value')
monthOrder = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
e_prc_melted['Month'] = pd.Categorical(e_prc_melted['Month'], categories=monthOrder, ordered=True)

## now sort the DataFrame by 'Year' and 'Month'
e_prc_melted = e_prc_melted.sort_values(by=['YYYY', 'Month'])

## combine Month and Year into one column
e_prc_melted['Time'] = e_prc_melted['Month'].astype(str) + '-' + e_prc_melted['YYYY'].astype(str)

# create the final 1D DataFrame with two desired columns and prc values
e_prc = e_prc_melted[['Time', 'Prc Value']].reset_index(drop=True)
e_prc['Prc Value'] = e_prc['Prc Value'].astype(float)

# Conversion
e_prc['Prc Value'] = e_prc['Prc Value'] * 0.001

# e_prc

In [130]:
####### RUNOFF #######

# Read the csv file and make 2000 the first year
filePath = '../Data/runoff_eri_arm.csv'
e_runoff = pd.read_csv(filePath, skiprows=2)
e_runoff = e_runoff.iloc[1224:, :3]
e_runoff = e_runoff.rename(columns={'Mon': 'Month'})
e_runoff = e_runoff.rename(columns={'Flow': 'Runoff Value (Flow)'})

# Adjsuting the way data frame is presented
## we want two columns: Time and Runoff Value

## melt the data frame and make 'Month' a categorical variable
month_conversion = {1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun', 7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'}
e_runoff['Month'] = e_runoff['Month'].map(month_conversion)

## combine Month and Year into one column
e_runoff['Time'] = e_runoff['Month'].astype(str) + '-' + e_runoff['Year'].astype(str)

# # create the final 1D DataFrame with two desired columns and runoff values
e_runoff = e_runoff[['Time', 'Runoff Value (Flow)']].reset_index(drop=True)
e_runoff = e_runoff.iloc[:-4, :]

# Conversion

e_SA_inv = (1 / (25.74 * 1e9))

e_runoff['Runoff Value (Flow)'] =e_runoff['Runoff Value (Flow)'].apply(lambda x: x * e_SA_inv)

e_runoff['Runoff Value (Flow)'] *= e_runoff['Time'].apply(get_time)

# e_runoff

In [131]:
s_erie = e_runoff.copy()
s_erie['Runoff Value (Flow)'] = e_prc.iloc[0:248, 1] - e_evap.iloc[0:248, 1] + e_runoff['Runoff Value (Flow)']
s_erie.rename(columns={'Runoff Value (Flow)':'Value'}, inplace=True)

In [132]:
s_erie

,Time,Value
0,Jan-2000,0.034867
1,Feb-2000,0.096071
2,Mar-2000,0.086858
3,Apr-2000,0.194553
4,May-2000,0.185830
...,...,...
243,Apr-2020,0.124855
244,May-2020,0.191417
245,Jun-2020,0.049215
246,Jul-2020,0.018314


### Lake Ontario


In [56]:
####### EVAPORATION #######

# Read the csv file and make 2000 the first year
filePath = '../Data/evaporation_ont.csv'
o_evap = pd.read_csv(filePath, skiprows=3)
o_evap = o_evap.iloc[50:, :]

# Adjsuting the way data frame is presented
## we want two columns: Time and Evap Value

## melt the data frame and make 'Month' a categorical variable
o_evap_melted = pd.melt(o_evap, id_vars=['YYYY'], var_name='Month', value_name='Evap Value')
monthOrder = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
o_evap_melted['Month'] = pd.Categorical(o_evap_melted['Month'], categories=monthOrder, ordered=True)

## now sort the DataFrame by 'Year' and 'Month'
o_evap_melted = o_evap_melted.sort_values(by=['YYYY', 'Month'])

## combine Month and Year into one column
o_evap_melted['Time'] = o_evap_melted['Month'].astype(str) + '-' + o_evap_melted['YYYY'].astype(str)

## create the final 1D DataFrame with two desired columns and evaporation values
o_evap = o_evap_melted[['Time', 'Evap Value']].reset_index(drop=True)

# Conversion
o_evap['Evap Value'] = o_evap['Evap Value'] * 0.001

# o_evap

In [57]:
####### PRECIPITATION #######

# Read the csv file and make 2000 the first year
filePath = '../Data/prc_ont_lake_mon.csv'
o_prc = pd.read_csv(filePath, skiprows=5)
o_prc = o_prc.iloc[60:81, :]

# Adjsuting the way data frame is presented
## we want two columns: Time and Prc Value

## melt the data frame and make 'Month' a categorical variable
o_prc_melted = pd.melt(o_prc, id_vars=['YYYY'], var_name='Month', value_name='Prc Value')
monthOrder = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
o_prc_melted['Month'] = pd.Categorical(o_prc_melted['Month'], categories=monthOrder, ordered=True)

## now sort the DataFrame by 'Year' and 'Month'
o_prc_melted = o_prc_melted.sort_values(by=['YYYY', 'Month'])

## combine Month and Year into one column
o_prc_melted['Time'] = o_prc_melted['Month'].astype(str) + '-' + o_prc_melted['YYYY'].astype(str)

# create the final 1D DataFrame with two desired columns and prc values
o_prc = o_prc_melted[['Time', 'Prc Value']].reset_index(drop=True)
o_prc['Prc Value'] = o_prc['Prc Value'].astype(float)

# Conversion
o_prc['Prc Value'] = o_prc['Prc Value'] * 0.001

# o_prc

In [133]:
####### RUNOFF #######

# Read the csv file and make 2000 the first year
filePath = '../Data/runoff_ont_arm.csv'
o_runoff = pd.read_csv(filePath, skiprows=2)
o_runoff = o_runoff.iloc[1224:, :3]
o_runoff = o_runoff.rename(columns={'Mon': 'Month'})
o_runoff = o_runoff.rename(columns={'Flow': 'Runoff Value (Flow)'})

# Adjsuting the way data frame is presented
## we want two columns: Time and Runoff Value

## melt the data frame and make 'Month' a categorical variable
month_conversion = {1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun', 7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'}
o_runoff['Month'] = o_runoff['Month'].map(month_conversion)

## combine Month and Year into one column
o_runoff['Time'] = o_runoff['Month'].astype(str) + '-' + o_runoff['Year'].astype(str)

# # create the final 1D DataFrame with two desired columns and runoff values
o_runoff = o_runoff[['Time', 'Runoff Value (Flow)']].reset_index(drop=True)
o_runoff = o_runoff.iloc[:-4, :]

# Conversion
o_SA_inv = (1 / (18.96 * 1e9))

o_runoff['Runoff Value (Flow)'] = o_runoff['Runoff Value (Flow)'].apply(lambda x: x * o_SA_inv)

o_runoff['Runoff Value (Flow)'] *= o_runoff['Time'].apply(get_time)

# o_runoff

In [134]:
s_ontario = o_runoff.copy()
s_ontario['Runoff Value (Flow)'] = o_prc.iloc[0:248, 1] - o_evap.iloc[0:248, 1] + o_runoff['Runoff Value (Flow)']
s_ontario.rename(columns={'Runoff Value (Flow)':'Value'}, inplace=True)

In [135]:
s_ontario

,Time,Value
0,Jan-2002,0.073649
1,Feb-2002,0.188194
2,Mar-2002,0.246815
3,Apr-2002,0.409382
4,May-2002,0.401872
...,...,...
219,Apr-2020,0.271037
220,May-2020,0.249169
221,Jun-2020,0.100589
222,Jul-2020,0.071239


### Lake Saint Clair


In [61]:
####### EVAPORATION #######

# Read the csv file and make 2000 the first year
filePath = '../Data/evaporation_stc.csv'
stc_evap = pd.read_csv(filePath, skiprows=3)
stc_evap = stc_evap.iloc[50:, :]

# Adjsuting the way data frame is presented
## we want two columns: Time and Evap Value

## melt the data frame and make 'Month' a categorical variable
stc_evap_melted = pd.melt(stc_evap, id_vars=['YYYY'], var_name='Month', value_name='Evap Value')
monthOrder = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
stc_evap_melted['Month'] = pd.Categorical(stc_evap_melted['Month'], categories=monthOrder, ordered=True)

## now sort the DataFrame by 'Year' and 'Month'
stc_evap_melted = stc_evap_melted.sort_values(by=['YYYY', 'Month'])

## combine Month and Year into one column
stc_evap_melted['Time'] = stc_evap_melted['Month'].astype(str) + '-' + stc_evap_melted['YYYY'].astype(str)

## create the final 1D DataFrame with two desired columns and evaporation values
stc_evap = stc_evap_melted[['Time', 'Evap Value']].reset_index(drop=True)

# Conversion
stc_evap['Evap Value'] = stc_evap['Evap Value'] * 0.001

# stc_evap

In [62]:
####### PRECIPITATION #######

# Read the csv file and make 2000 the first year
filePath = '../Data/prc_stc_lake_mon.csv'
stc_prc = pd.read_csv(filePath, skiprows=5)
stc_prc = stc_prc.iloc[60:81, :]

# Adjsuting the way data frame is presented
## we want two columns: Time and Prc Value

## melt the data frame and make 'Month' a categorical variable
stc_prc_melted = pd.melt(stc_prc, id_vars=['YYYY'], var_name='Month', value_name='Prc Value')
monthOrder = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
stc_prc_melted['Month'] = pd.Categorical(stc_prc_melted['Month'], categories=monthOrder, ordered=True)

## now sort the DataFrame by 'Year' and 'Month'
stc_prc_melted = stc_prc_melted.sort_values(by=['YYYY', 'Month'])

## combine Month and Year into one column
stc_prc_melted['Time'] = stc_prc_melted['Month'].astype(str) + '-' + stc_prc_melted['YYYY'].astype(str)

# create the final 1D DataFrame with two desired columns and prc values
stc_prc = stc_prc_melted[['Time', 'Prc Value']].reset_index(drop=True)
stc_prc['Prc Value'] = stc_prc['Prc Value'].astype(float)

# Conversion
stc_prc['Prc Value'] = stc_prc['Prc Value'] * 0.001

# stc_prc

In [136]:
####### RUNOFF #######

# Read the csv file and make 2000 the first year
filePath = '../Data/runoff_stc_arm.csv'
stc_runoff = pd.read_csv(filePath, skiprows=2)
stc_runoff = stc_runoff.iloc[816:, :3]
stc_runoff = stc_runoff.rename(columns={'Mon': 'Month'})
stc_runoff = stc_runoff.rename(columns={'Flow': 'Runoff Value (Flow)'})

# Adjsuting the way data frame is presented
## we want two columns: Time and Runoff Value

## melt the data frame and make 'Month' a categorical variable
month_conversion = {1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun', 7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'}
stc_runoff['Month'] = stc_runoff['Month'].map(month_conversion)

## combine Month and Year into one column
stc_runoff['Time'] = stc_runoff['Month'].astype(str) + '-' + stc_runoff['Year'].astype(str)

# # create the final 1D DataFrame with two desired columns and runoff values
stc_runoff = stc_runoff[['Time', 'Runoff Value (Flow)']].reset_index(drop=True)
stc_runoff = stc_runoff.iloc[:-4, :]

# Conversion
stc_SA_inv = (1 / (1.114 * 1e9 ))

stc_runoff['Runoff Value (Flow)'] = stc_runoff['Runoff Value (Flow)'].apply(lambda x: x * stc_SA_inv)

stc_runoff['Runoff Value (Flow)'] *= stc_runoff['Time'].apply(get_time)

# stc_runoff

In [137]:
s_clair = stc_runoff.copy()
s_clair['Runoff Value (Flow)'] = stc_prc.iloc[0:248, 1] - stc_evap.iloc[0:248, 1] + stc_runoff['Runoff Value (Flow)']
s_clair.rename(columns={'Runoff Value (Flow)':'Value'}, inplace=True)

In [138]:
s_clair

,Time,Value
0,Jan-2000,0.171952
1,Feb-2000,0.370971
2,Mar-2000,0.267730
3,Apr-2000,0.567235
4,May-2000,0.489634
...,...,...
243,Apr-2020,0.257870
244,May-2020,0.373671
245,Jun-2020,0.088942
246,Jul-2020,-0.050689


In [140]:
s_superior = s_superior.iloc[36:, :].reset_index(drop=True)
s_clair = s_clair.iloc[36:, :].reset_index(drop=True)
s_erie = s_erie.iloc[36:, :].reset_index(drop=True)
s_ontario = s_ontario.iloc[12: , :].reset_index(drop=True)

In [141]:
print('Superior total entries', s_superior.shape[0])
print('MH total entries', s_mh.shape[0])
print('St. Clair total entries', s_clair.shape[0])
print('Erie total entries', s_erie.shape[0])
print('Ontario total entries', s_ontario.shape[0])

Superior total entries 212
MH total entries 212
St. Clair total entries 212
Erie total entries 212
Ontario total entries 212


In [142]:
exogenous_time_vectors = []
exogenous_time_vectors_short = []
for t in range(0, s_superior.shape[0]):
    s_t = np.array([
            s_superior['Value'][t],
            s_mh['Value'][t],
            s_clair['Value'][t],
            s_erie['Value'][t],
            s_ontario['Value'][t]
        ])
    
    year = int(s_superior['Time'][t][4:9])
    if year >= 2012: 
        exogenous_time_vectors_short.append(s_t)
    
    exogenous_time_vectors.append(s_t)

In [146]:
np.save('../Data/all-exogenous.npy', exogenous_time_vectors)
np.save('../Data/exogenous_2012_forward.npy', exogenous_time_vectors_short)